In [10]:
import pandas as pd

pd.set_option('display.max_colwidth', None)
df = pd.read_csv('processed_zillow_reviews.csv') 

#I'll sample proportionally based on existing data distribution
target_size = 10000

sampled_df = df.groupby('Rating').apply(
    lambda x: x.sample(frac=target_size / len(df), random_state=42)
).reset_index(drop=True)

data = sampled_df['Cleaned_Description'].dropna().astype(str).tolist()



/tmp/ipykernel_43841/2647810879.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_df = df.groupby('Rating').apply(


In [11]:
#tokenize pre-processed data (split into word)
tokenized_data = [text.split() for text in data]

In [12]:
#Preparing the data. LDA requires Corpus and Dictionary
from gensim.corpora.dictionary import Dictionary

# Create dictionary, each word will have a unique ID
id2word = Dictionary(tokenized_data)

# Filter extreme cases (optional, improves quality)  Remove words that appear in fewer than 10 documents and more than 50% of documents
id2word.filter_extremes(no_below=10, no_above=0.5)

# Create bag-of-words corpus
corpus = [id2word.doc2bow(text) for text in tokenized_data]

corpus

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 2),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 2),
  (28, 1),
  (29, 2),
  (30, 2),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 2),
  (35, 1),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 2),
  (43, 1),
  (44, 1)],
 [(4, 1),
  (10, 1),
  (30, 2),
  (42, 1),
  (44, 2),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1)],
 [(0, 1),
  (15, 2),
  (23, 1),
  (34, 1),
  (55, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 1),
  (80, 1),
  (81, 1)

In [46]:
from gensim.models import LdaModel

# Train LDA model
lda_model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    num_topics=40,  # change to 6, 10, etc. to experiment
    random_state=42,
    update_every=1,
    chunksize=100,
    passes=10,
    alpha='auto',
    per_word_topics=True
)

In [47]:
# Print the keywords in each topic
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic {idx + 1}: {topic}\n")


Topic 1: 0.375*"amazing" + 0.169*"professionalism" + 0.129*"high" + 0.082*"bid" + 0.057*"calm" + 0.041*"selling" + 0.028*"condition" + 0.026*"coordinate" + 0.018*"paper" + 0.014*"fashion"

Topic 2: 0.453*"extremely" + 0.163*"long" + 0.111*"life" + 0.095*"situation" + 0.067*"la" + 0.052*"enjoy" + 0.038*"30" + 0.000*"mission" + 0.000*"vendor" + 0.000*"simplify"

Topic 3: 0.156*"call" + 0.104*"phone" + 0.081*"mark" + 0.064*"second" + 0.061*"customer" + 0.054*"far" + 0.051*"later" + 0.048*"importantly" + 0.042*"pick" + 0.042*"minute"

Topic 4: 0.270*"show" + 0.104*"mortgage" + 0.082*"range" + 0.071*"weekend" + 0.068*"connect" + 0.050*"soon" + 0.042*"will" + 0.042*"tirelessly" + 0.041*"stop" + 0.038*"caring"

Topic 5: 0.124*"work" + 0.122*"buy" + 0.107*"great" + 0.095*"highly" + 0.091*"help" + 0.090*"experience" + 0.067*"knowledgeable" + 0.049*"patient" + 0.044*"easy" + 0.038*"responsive"

Topic 6: 0.318*"lot" + 0.134*"john" + 0.109*"ready" + 0.083*"let" + 0.078*"thankful" + 0.057*"drive" +

In [48]:
import pyLDAvis.gensim_models
import pyLDAvis

# Visualize topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
vis

# Ask prof if these topics makes sense? what is her input on this topic modelling?

#top 6 words in Topic 0 - help, work, process, time, find, buy
#top 6 words in Topic 1 - house, offer, property, want, get, area
#top 6 words in Topic 2 - agent, know, real, estate, good, like
#top 6 words in Topic 3 - market, knowledge, expertise, negotiation, ensure, professionalism
#top 6 words in Topic 4 - provide, service, keep, hard, excellent, advice

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
34    -0.048109 -0.322810       1        1  33.996878
15     0.091097 -0.223165       2        1  10.738612
4      0.103089 -0.341135       3        1  10.134407
11     0.000685 -0.043003       4        1   5.259334
21     0.051063 -0.026492       5        1   4.957775
33     0.111977  0.008193       6        1   2.396612
17     0.111473  0.009711       7        1   2.110144
24     0.067738 -0.022241       8        1   2.038361
7      0.099388  0.011961       9        1   1.951201
18     0.030319  0.020743      10        1   1.668440
10     0.067886  0.021826      11        1   1.585571
27     0.015316  0.035642      12        1   1.285155
2      0.035577  0.027660      13        1   1.253227
6      0.033391  0.031481      14        1   1.211110
25     0.084713  0.028360      15        1   1.188130
19     0.057191  0.031623      16        1   1.132454
0      0.054468  0.031756      17        1   1.125307
3      0.036675  0.034012      18        1   1.050636
8      0.047447  0.033522      19        1   1.046228
1      0.054214  0.037644      20        1   1.008638
16     0.014103  0.032434      21        1   0.999736
37     0.053722  0.035153      22        1   0.990482
5      0.048572  0.034073      23        1   0.964073
9      0.036824  0.034291      24        1   0.964020
29     0.029778  0.037022      25        1   0.933544
30     0.037445  0.037570      26        1   0.896239
28     0.033939  0.038148      27        1   0.889773
38     0.061222  0.037192      28        1   0.885347
13     0.043744  0.038462      29        1   0.873352
32     0.049906  0.039370      30        1   0.868114
12     0.051515  0.040243      31        1   0.852520
35     0.017141  0.041738      32        1   0.767120
20     0.004347  0.043650      33        1   0.702384
14    -0.044728  0.053001      34        1   0.488704
26    -0.028609  0.053360      35        1   0.427293
23    -0.068196  0.057789      36        1   0.333778
39    -0.361581 -0.009696      37        1   0.006325
31    -0.361581 -0.009696      38        1   0.006325
36    -0.361581 -0.009696      39        1   0.006325
22    -0.361581 -0.009696      40        1   0.006325, topic_info=            Term         Freq        Total Category  logprob  loglift
250        house  5927.000000  5927.000000  Default  30.0000  30.0000
79       process  7567.000000  7567.000000  Default  29.0000  29.0000
86          time  6306.000000  6306.000000  Default  28.0000  28.0000
44          work  8155.000000  8155.000000  Default  27.0000  27.0000
188     question  3223.000000  3223.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
24   negotiation     0.009493   805.521491  Topic40  -7.7911  -1.6803
25      neighbor     0.009493    76.131291  Topic40  -7.7911   0.6788
26        person     0.009493   887.528671  Topic40  -7.7911  -1.7772
27         point     0.009493   518.159660  Topic40  -7.7911  -1.2390
29       problem     0.009493   232.086054  Topic40  -7.7911  -0.4359

[1792 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
753       3  0.996557     100
953      21  0.989668     1st
431      16  0.998379       2
126      13  0.984749      20
370      12  0.977419     2nd
...     ...       ...     ...
213       8  0.994510   write
736      36  0.973339       y
214       1  0.998970    year
360      12  0.981645    zero
215      23  0.990284  zillow

[684 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[35, 16, 5, 12, 22, 34, 18, 25, 8, 19, 11, 28, 3, 7, 26, 20, 1, 4, 9, 2, 17, 38, 6, 10, 30, 31, 29, 39, 14, 33, 13, 36, 21, 15, 27, 24, 40, 32, 37, 23])

In [16]:
pyLDAvis.save_html(vis, 'lda_visualization_5_topics.html')


In [11]:
'''
1)  Each circle represents a topic.
    Size of circle = how prevalent (dominant) that topic is in your dataset.
    Distance between circles = how different the topics are in terms of vocabulary.

2)  Saliency (blue bars):
    How much this word contributes to distinguishing topics.
    High-saliency words are useful for identifying topics.
    
    Red bars (when a topic is selected):
    How frequent this word is within the selected topic.
    You’ll see these when you click on a topic bubble (like Topic 1, 2, etc.).

3)  
'''

UsageError: Cell magic `%%htmlEach` not found.


In [17]:
#assigning topics to ratings
def get_dominant_topic(bow):
    topics = lda_model.get_document_topics(bow)
    topics = sorted(topics, key=lambda x: x[1], reverse=True)
    return topics[0][0] if topics else None  # topic number

# Apply to your reviews
sampled_df['Dominant_Topic'] = [get_dominant_topic(doc) for doc in corpus]


In [29]:
# Check a few rows
sampled_df['Dominant_Topic'].unique()


array([2, 3, 0, 4])

In [31]:
# Count of reviews per (Rating, Topic_Label) pair
topic_rating_counts = sampled_df.groupby(['Rating', 'Dominant_Topic']).size().reset_index(name='Count')

# Get the most frequent topic for each rating
dominant_topic_per_rating = topic_rating_counts.loc[
    topic_rating_counts.groupby('Rating')['Count'].idxmax()
].reset_index(drop=True)


#This shows the the topic that was mostly assigned to a Rating
print(dominant_topic_per_rating)


   Rating  Dominant_Topic  Count
0       1               2      8
1       2               2      1
2       3               0      3
3       4               0     44
4       5               0   9017
